In [1]:
import string

In [2]:
class Enigma:
  def __init__(self, rotor_settings=None, reflector_letter="B", plugboard_settings=None):
    if rotor_settings == None:
      rotor_settings = [('III', 0), ('II', 0), ('I', 0)]
    if plugboard_settings == None:
      plugboard_settings = []

    self.rotors = [Rotor(rotor_number, ring_offset) for rotor_number, ring_offset in rotor_settings]
    self.reflector = Reflector(reflector_letter)
    self.plugboard = Plugboard(plugboard_settings)

  """ Resets the Enigma machine """
  def reset(self):
    for rotor in self.rotors:
      rotor.reset()

  """ Encodes plaintext """
  def encode_plaintext(self, plaintext):
    plaintext = plaintext.upper()
    if not plaintext.isalpha():
      print("Invalid plaintext!")
      pass
    
    ciphertext = ""
    for char in plaintext:
      ciphertext += self.encode(char)

    return ciphertext

  """ Encodes one character """
  def encode(self, char):
    # Normal Step
    for rotor in self.rotors:
      # Rotate the next rotor if it hits a turnover
      if (not rotor.rotate()):
        break

    # Double Step
    for turnover in self.rotors[1].turnovers:
      # the subsequent character mod len(alphabet)
      notch = string.ascii_uppercase[(string.ascii_uppercase.index(turnover) + 1) % len(string.ascii_uppercase)]
      if string.ascii_uppercase[self.rotors[1].number_of_rotates % len(string.ascii_uppercase)] == notch:
        # Rotate the third rotor if the second rotor hits a turnover
        if(self.rotors[1].rotate()):
          self.rotors[2].rotate()

    # Input character, goes through the plugboard, and returns an index
    index = self.plugboard.forward(char)

    # Input index to rotors from right to left
    for rotor in self.rotors:
      index = rotor.forward(index)

    # Input index into the reflector
    index = self.reflector.forward(index)

    # Input character, goes through the plugboard, and returns an index
    for rotor in reversed(self.rotors):
      index = rotor.reverse(index)

    # Input index, goes through the plugboard, and returns a character
    return self.plugboard.reverse(index)

class Rotor:
  def __init__(self, rotor_number, ring_offset):
    self.rotor_number = rotor_number
    self.settings = {
      "I": { "sequence": "EKMFLGDQVZNTOWYHXUSPAIBRCJ", "turnovers": ["R"]},
      "II": { "sequence": "AJDKSIRUXBLHWTMCQGZNPYFVOE", "turnovers": ["F"]},
      "III": { "sequence": "BDFHJLCPRTXVZNYEIWGAKMUSQO", "turnovers": ["W"]},
      "IV": { "sequence": "ESOVPZJAYQUIRHXLNFTGKDCMWB", "turnovers": ["K"]},
      "V": { "sequence": "VZBRGITYUPSDNHLXAWMJQOFECK", "turnovers": ["A"]},
      "VI": { "sequence": "JPGVOUMFYQBENHZRDKASXLICTW", "turnovers": ["A", "N"]},
      "VII": { "sequence": "NZJHGRCXMYSWBOUFAIVLPEKQDT", "turnovers": ["A", "N"]},
      "VIII": { "sequence": "FKQHTLXOCBJSPDZRAMEWNIUYGV", "turnovers": ["A", "N"]},
    }
    self.sequence = self.settings[self.rotor_number]["sequence"]
    self.turnovers = self.settings[self.rotor_number]["turnovers"]
    self.ring_offset = ring_offset
    self.number_of_rotates = 0

    for _ in range(self.ring_offset):
      self.rotate()

  """ Resets the rotors to the initial state """
  def reset(self):
    self.number_of_rotates = 0
    self.sequence = self.settings[self.rotor_number]["sequence"]
    self.turnovers = self.settings[self.rotor_number]["turnovers"]

    for _ in range(self.ring_offset):
      self.rotate()

  """ Returns the index of the character, in respect to the alphabet, after passing through the sequence """
  def reverse(self, index):
    return string.ascii_uppercase.index(self.sequence[index])

  """ Returns the index of the character, in respect to the sequence, after passing through the alphabet """
  def forward(self, index):
    return self.sequence.index(string.ascii_uppercase[index])


  """ Rotates the rotors """
  def rotate(self):
    self.number_of_rotates += 1
    self.sequence = self.sequence[1:] + self.sequence[:1]
    for char in self.turnovers:
      if char == string.ascii_uppercase[self.number_of_rotates % len(string.ascii_uppercase)]:
        return True
    return False

class Reflector:
  def __init__(self, reflector_letter):
    settings = {
      "A": "EJMZALYXVBWFCRQUONTSPIKHGD",
      "B": "YRUHQSLDPXNGOKMIEBFZCWVJAT",
      "C": "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    }
    self.sequence = settings[reflector_letter]

  """ Returns the index of the character, in respect to the sequence, after passing through the alphabet """
  def forward(self, index):
    return self.sequence.index(string.ascii_uppercase[index])

class Plugboard:
  def __init__(self, mapping):
    # mapping = [("A", "B"), ("C", "D")]
    self.mapping = {}

    for char in string.ascii_uppercase:
      self.mapping[char] = char

    for a, b in mapping:
      self.mapping[a] = b
      self.mapping[b] = a

  """ Returns the index of the mapped character """
  def forward(self, char):
    return string.ascii_uppercase.index(self.mapping[char])

  """ Returns the character of the index """
  def reverse(self, index):
    return self.mapping[string.ascii_uppercase[index]]

In [3]:
enigma = Enigma()
enigma.encode_plaintext('EVAN')

'NOVD'

In [4]:
enigma.reset()
enigma.encode_plaintext('NOVD')

'EVAN'